# Predictive Modelling

## Import dependencies

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import statsmodels
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

%matplotlib inline

# A classification problem : the Adult Income dataset

Another common use case in Supervised Learning is classification. Let's try one example now with the **Adult Income** dataset. This dataset was extracted from the 1994 Census database and is described by the following variables:

- **age**: continuous.
- **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- **fnlwgt**: continuous.
- **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- **education-num**: continuous.
- **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- **sex**: Female, Male.
- **capital-gain**: continuous.
- **capital-loss**: continuous.
- **hours-per-week**: continuous.
- **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- **income**: `>50K`, `<=50K`

Prediction task is to determine whether a person makes over 50K a year.

## Recap of processing data from last week

Recap: 
* check for and drop meaningless variables
* explore our data - are there any variables we should remove?
* one-hot encoding of variables
* build dummy variables from features we have already
* remove hard to interpret variables

In [ ]:
# Upload the data 
df_raw = pd.read_csv("adult.csv", index_col=[0])

# Drop meaningless variables
df_raw = df_raw.drop(['fnlwgt'], axis=1)

In [ ]:
# Majority of United-States occurrences
df_raw = df_raw.drop(['native.country'], axis=1)

# One-hot encode categorical variables 
one_hot_df = pd.get_dummies(df_raw, drop_first=True)


Let's take a look at the new table 

In [ ]:
one_hot_df.head()

In [ ]:
# Build dummy variable for working vs not working ('Without-pay')
for col in one_hot_df.columns:
    if 'workclass_' in col and ' Without-pay' not in col:
        one_hot_df = one_hot_df.drop([col], axis=1)

In [ ]:
# Remove education.num as it is difficult to interpret (and we have other columns for education levels)
# Please complete this line
one_hot_df = 

# Predictive modelling <a name="Modelling"></a>

The predictive analysis follows a descriptive stage, once we have:
 - cleaned the data
 - created a few derived variables
 - have a better understanding of the data. 
 
Its aim is to build a performing predictive model and to identify the main explanatory variables. This is an iterative process as a variety of modelling approaches are usually tested and new features are created.

The primary goal of this section is to accurately predict whether an adult is going to make more than 50K a year and understand the drivers explaining such an income.

## Exploring the data

We can look at the proportion of positive and negative labels i.e. the number of rows with income > 50K vs the number of rows where it is not. 


In [ ]:
#Print the number of positive and negative rows below:


We have a reasonable amount of data in each class which is good. Sometimes we can have imbalanced classes, where the size of one class is much smaller than the size of the other class. In this case, we may need some additional methods to address this. See https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/ for more information if you are interested. 

### Train/test split <a name="Train_test_split"></a>

A good prediction model must be generalisable - i.e., it should be able to make accurate predictions on new data. Several methods exist, to make our model more generalisable, of which the 'train/test split'.

Other methods, such as train/test/validation split and cross-validation take this approach one step further.

In [ ]:
# Remove the target variable from the dataset
target = one_hot_df['income_>50K']
one_hot_df_indep = one_hot_df.drop('income_>50K', axis=1)

In [ ]:
# Splitting the data into training and testing sets
# a frequent test set size is 20%-30% of the original dataset

X = one_hot_df_indep.values
X_train, X_test, y_train, y_test  = train_test_split(X, target, test_size = 0.2)

### Decision tree <a name="Decision_tree"></a>

A decision tree can be perceived as a set of rules which enable to better describe and predict a given phenomenon, e.g., here, a salary `>50K` or `<= 50K`. As its name indicates, a tree is composed of branches (which link the nodes to one another or to the final leaves), nodes (in the middle of the tree) and leaves (higher end of the tree).

In our case, the tree splits at each node, on a **rule/condition** (explanatory variable and a set of values - e.g., **age > 38**) according to a criterion (Gini), to better separate `>50K` vs `<= 50K` populations.

The decision tree algorithm has several important hyperparameters (cf. below). Understanding these will enable you to avoir overfitting:

* the splitting criterion, for each node: Gini (most frequently used), entropy etc.
* the maximum depth of the tree: how many branches link the first node to the end leaves?
* the minimum sample split: the minimum number of data points in each node, after a split
* the minimum samples per leaf: the minimum number of data points in each final leaf
* the maximum number of features to consider when looking for the best split *(from the sklearn documentation)*:
    * If int, then consider max_features features at each split.
    * If float, then max_features is a percentage and int(max_features * n_features) features are considered at each split.
    * If “auto”, then max_features=sqrt(n_features).
    * If “sqrt”, then max_features=sqrt(n_features).
    * If “log2”, then max_features=log2(n_features).
    * If None, then max_features=n_features.
    
For a more in depth explanation of how to tune these parameters, please refer to https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680 .

In [ ]:
# Decision tree
# Initiate your Decision Tree Classifier model
clf = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_split=20, 
                             min_samples_leaf=10, max_features=None)

# Fit your Decision Tree model to your train model
clf.fit(X_train, y_train)

# Predict labels on your Test set of independent variables
pred_clf = clf.predict(X_test)
# Predict probabilities on your Test set of independent variables
proba_clf = clf.predict_proba(X_test)

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred_clf))

We guess the labels correctly 85% of the time! But, can we think of a situation when accuracy is not a good measure of predictive power?

In [ ]:
#Can also take a look at the confusion matrix, to see where our classifier is making mistakes
print(metrics.confusion_matrix(y_test, pred_clf))

In [ ]:
# Other evaluation metrics...
print(metrics.classification_report(y_test, pred_clf))

auc = metrics.roc_auc_score(y_test, proba_clf[:,1])
print("AUC: ", auc)

### Feature importance

In [ ]:
# Plot feature importance
features_log = pd.DataFrame(clf.feature_importances_, index = one_hot_df_indep.columns.tolist(), 
                            columns = ['Importance'])
features_log = features_log.sort_values(by='Importance', ascending=True)

features_log.plot(kind='barh', figsize=(12,9), color = 'blue')
plt.xlabel('Feature importance')
plt.title('Feature importance for Decision Tree')
plt.show()

## Advanced (aka I'm feeling smart)

#### Boosting vs bagging

* In this section, we are going to explore new types of models (random forests and XGBoost), which are globally called 'ensemble methods'. 

* The idea behind ensemble methods is to combine several models to have a better predictive performance. E.g., a single decision tree, is a weak model. If we build several decision trees on samples of the data, the final model will be more robust to outliers and will hence have a better predictive performance. 

* To understand this concept, we need to have a look at bagging and boosting, which are two sampling methods used in ensemble methods.

* **Bagging** consists in sampling with replacement. Ensemble methods using the bagging approach, build models on each sample of data; the final score of the ensemble model will be a *vote* for classification and an average for regression.

* In **Boosting**, data points all have equal weights at the beginning. These weights are increased, if the data points are misclassified (the algorithm will focus on them) and decreased, if they are well classified. 

### Random Forest <a name="Random_forest"></a>

A Random forest is an example of bagging, and consists of an ensemble of decision trees 
(built on subsets of the data, which are sampled with replacement). 
Tree nodes are split on a random subset of features. The amount of features selected is 
one of the hyperparameters of the model (max_features, here).

NB: In general, a smaller subset of variables produces less correlation and as such, a lower error rate.
There a few rules of thumb:

If M is the number of features in the dataset and m is the max number of features randomly sampled:

m << M

- for regression: m = M/3
- for classification: m = sqrt(M)

For a more advanced read on how to tune the hyperparameters of a random forest, please see https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 .

In [ ]:
# Initiate the Random Forest Classifier object with the chosen hyperparameters
clf_rf = RandomForestClassifier(random_state = 33, n_estimators=50, max_depth=5, min_samples_leaf=30, max_features=4)

In [ ]:
# Fit the Random Forest to your train data below


In [ ]:
# Predict labels on your Test set of independent variables
pred_rf = 
# Predict probabilities on your Test set of independent variables
proba_rf = 

In [ ]:
# Print out the model accuracy below
print("Accuracy:", #put your answer here )

In [ ]:
#Lets take a look again at the confusion matrix, to see where our classifier is making mistakes
print(metrics.confusion_matrix(y_test, pred_rf))

In [ ]:
# Other evaluation metrics...
print(metrics.classification_report(y_test, pred_rf))

auc = metrics.roc_auc_score(y_test, proba_rf[:,1])
print("AUC: ", auc)

In [ ]:
# Plot feature importance
features_log = pd.DataFrame(clf_rf.feature_importances_, index = one_hot_df_indep.columns.tolist(), 
                            columns = ['Importance'])
features_log = features_log.sort_values(by='Importance', ascending=True)

features_log.plot(kind='barh', figsize=(12,9), color = 'blue')
plt.xlabel('Feature importance')
plt.title('Feature importance for Random Forest')
plt.show()

## Some homework...

For some additional work, I would first recommend reading through some of the links suggested in this notebook. These will give you a thorough understanding of how to apply these methods in practice, and will be relevant for your final project.

### Hyperparameter tuning task: (Advanced)

As previously mentioned, it is important to choose 'good' parameters for a random forest, based on the data. One method of doing this is by using cross-validation. In cross-validation, in addition to our usual train and test split of the data we further split our training set into K subsets, called folds. We can then train our Random Forest on K-1 folds and test on the remaining one, repeating to use each fold as the test. We do this for each set of parameters we want to search over, and average the result over the folds. 

Let's have a go at this....

Use https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 as a resource to fill in the gaps yourself. 

In [ ]:
# Take a look at the parameters of that we used before
pprint(clf_rf.get_params())

In [ ]:
# Let's search over the following parameter settings #TODO: edit this 

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 150, num = 11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 55, num = 5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [30, 40, 50]
# Method of selecting samples for training each tree
bootstrap = [True]

In [ ]:
# Complete the random grid dictionary with all of the parameters we are searching over: 
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               #complete this here 
               }

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model, similarly to how we did before:
rf = ...

# Declare the random search using 3 fold cross validation, searching over 100 combinations of the parameters 
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)


In [ ]:
# Now fit the random search model with our training data below


In [ ]:
# What is the best choice of hyperparameters found by the random search? Do this below


In [ ]:
# Let's find out how the accuracy of this model with tuned hyperparameters compares! 
# Predict the labels on your test set, as we did before, and then print the model accuracy


Did you improve the accuracy? What a difference the parameters of the model can make!